# Regression chain graph for GSS data

## Read the data

In [34]:
library("ggm")
library("cta")
source("funs_RCG.R")

In [35]:
ex <- read.csv("gss.txt", na.string = c("0", "8", "9"))
year <- ex$YEAR
ex$SATJOB[ex$SATJOB == 4] <- 3
ex <- (ex[c(3, 4, 5, 6, 7, 8)]) # CAPPUN(D),GUNLAW(G),SEX(S),ABRAPE(A),CONFINAN(F),SATJOB(J)
names(ex) <- c("D", "G", "S", "A", "B", "J")
gss_raw <- ex

In [36]:
head(gss_raw)

,D,G,S,A,B,J
,<int>,<int>,<int>,<int>,<int>,<dbl>
1,1,2,2,2,1,1
2,1,NA,1,NA,1,1
3,2,NA,2,NA,2,NA
4,2,1,2,1,NA,2
5,2,1,1,1,NA,NA
6,1,NA,2,NA,NA,1


In [56]:
gss_data4 <- na.omit(gss_raw[, c("D", "G", "J", "B")])
for (j in 1:4) gss_data4[, j] <- factor(gss_data4[, j])
tab <- table(gss_data4)
ft <- ftable(B + J ~ G + D, tab)

In [38]:
dim(gss_data4)

[1] 13473     4

In [39]:
ft

    B    1              2              3          
    J    1    2    3    1    2    3    1    2    3
G D                                               
1 1   1168  705  254 1931 1577  548  449  367  218
  2    453  299  112  733  671  300  201  175   88
2 1    389  242   83  663  540  184  158  165   74
  2    107   75   26  159  153   71   51   51   33

## Define the link function

Vector of joint frequencies

In [58]:
y <- as.vector(tab)

In [59]:
c1 <- c(1, 2)
c2 <- c(3, 4)

In [60]:
b <- c(2,2,3,3)
P <- powerset(seq_along(b[c1]))
P

[[1]]
[1] 1

[[2]]
[1] 2

[[3]]
[1] 1 2

In [61]:
Clist <- ctr_list(P, b[c1])
print(Clist)

[[1]]
     [,1] [,2]
[1,]   -1    1

[[2]]
     [,1] [,2]
[1,]   -1    1

[[3]]
     [,1] [,2] [,3] [,4]
[1,]    1   -1   -1    1



In [62]:
link1 <- function(y) mlogit_link(y, b[c1], P, Clist)

Responses D G, Explanatory variables J B

In [63]:
XJB <- expand.grid(
  J = factor(1:3, levels = c("1", "2", "3")),
  B = factor(1:3, levels = c("1", "2", "3"))
)

In [64]:
ZD <- model.matrix(~ J * B, XJB)[, ]
ZG <- model.matrix(~ J * B, XJB)[, ]
ZDG <- model.matrix(~ J * B, XJB)[, ]

Z1 <- blkdiag(ZD, ZG, ZDG)

Maximum likelihood fit of the saturated model `(D*G) ~ (J*B)`

In [66]:
out <- mph.fit(y = y, L.fct = link1, X = Z1)
out$Gsq

,LIKELIHOOD RATIO STATISTIC
,0


Estimates

In [71]:
be <- round(out$beta, 3)
be_se <- round(sqrt(diag(out$covbeta)), 2)
pval <- round(2 * (1 - pnorm(abs(be / be_se))), 3)
h <- cbind(be, be_se, pval)
colnames(h) <- c("beta", "se", "p")
ta <- cbind(h[1:9, 1:3], h[10:18, 1:3], h[19:27, 1:3])
ta

,beta,se,p,beta,se,p,beta,se,p
(Intercept),-1.023,0.05,0.000,-1.184,0.05,0.000,-0.344,0.12,0.004
J2,0.094,0.08,0.240,0.031,0.08,0.698,0.030,0.19,0.875
J3,0.130,0.11,0.237,-0.027,0.12,0.822,0.002,0.28,0.994
B2,-0.045,0.06,0.453,0.008,0.06,0.894,-0.116,0.16,0.468
B3,0.143,0.09,0.112,0.050,0.09,0.579,0.017,0.22,0.938
J2:B2,0.030,0.10,0.764,-0.032,0.10,0.749,0.023,0.24,0.924
J3:B2,0.258,0.13,0.047,0.001,0.15,0.995,0.108,0.34,0.751
J2:B3,-0.071,0.13,0.585,0.183,0.14,0.191,-0.136,0.32,0.671
J3:B3,-0.132,0.17,0.437,0.111,0.18,0.537,0.425,0.41,0.300
